In [1]:
import pymrio
import pandas as pd
import pickle

In [3]:
io = pymrio.parse_exiobase3('C://Users/Maxime/Desktop/Thesis/Bdd/EXIOBASE/monetary/exiobase3.4/IOT_2011_pxp/')

In [5]:
new_a = io.A.copy()
new_a = pd.concat([new_a,new_a.loc[:,[i for i in io.A.columns if i[0] == 'CA']]],axis=1)
new_a = pd.concat([new_a,new_a.loc[[i for i in io.A.columns if i[0] == 'CA'],:]],axis=0)

In [7]:
countries = [i[0] for i in new_a.iloc[::200].index]
countries[-1]='QC'

In [8]:
new_a.index = pd.MultiIndex.from_product([countries,[i[1] for i in new_a.iloc[:200].index]],names=('region', 'sector'))
new_a.columns = pd.MultiIndex.from_product([countries,[i[1] for i in new_a.iloc[:200].index]],names=('region', 'sector'))

In [10]:
new_a[('QC','grid_mix')] = pd.Series(0, new_a.index)

In [12]:
new_a = pd.concat([new_a,pd.DataFrame(0,pd.MultiIndex.from_product([['QC'],['grid_mix']],names=('region', 'sector')),new_a.columns)],axis=0)

In [14]:
# change Quebec grid mix here if you need (give %)
grid_mix_quebec = {'Electricity by coal':0,'Electricity by gas':0,'Electricity by nuclear':0,'Electricity by hydro':99.1,'Electricity by wind':0.5,
                  'Electricity by petroleum and other oil derivatives':0.2,'Electricity by biomass and waste':0.2,'Electricity by solar photovoltaic':0,
                  'Electricity by solar thermal':0,'Electricity by tide, wave, ocean':0,'Electricity nec':0}

In [15]:
for technology in grid_mix_quebec.keys():
    new_a.loc[('QC',technology),('QC','grid_mix')] = grid_mix_quebec[technology]/100

In [20]:
new_a.loc[('QC','grid_mix'),[i for i in new_a.columns if 'QC' == i[0]]] = new_a.loc[[i for i in new_a.index if 'Electricity' in i[1] and 'CA' == i[0]], [i for i in new_a.columns if 'QC' == i[0]]].sum()

In [23]:
# international trade only buys products from Canada (and not from Quebec) [apply one or the other]
new_a.loc[[i for i in new_a.index if i[0] == 'QC'],[i for i in new_a.columns if i[0] != 'QC']] = 0

In [24]:
# international trade only buys products from Quebec (and not from Canada) [apply one or the other]
new_a.loc[[i for i in new_a.index if i[0] == 'CA'],[i for i in new_a.columns if i[0] != 'CA']] = 0

In [25]:
# no interprovince trade
new_a.loc[[i for i in new_a.index if i[0] == 'CA'],[i for i in new_a.columns if i[0] == 'QC']] = 0

In [26]:
with open('C://Users/Maxime/Desktop/Thesis/BD_conso_QC/technology_matrix.pickle','wb') as f:
    pickle.dump(new_a, f)